In [1]:
import numpy as np
import sys
import matplotlib.pyplot as plt
import h5py 
sys.path.append('/Users/ajc/Core/PostProcessData/PyScript/')
import readdata as rd

In [2]:
path = '/Users/ajc/Core/Projects/ATS-Data/OR-CONDO/simulations/hillslope/2019/MaxSnow/'
infile = ['MS_B_1R1', 'MS_S30_1R1', 'MS_S60_1R1', 'MS_S90_1R1']

infile = [path + file +'/snow-depth-max.dat' for file in infile]

#infile = [path + file +'/surface_flux-day.dat' for file in infile]

infile_forcing = ['snowforcing/Barrow-20yrs-maxsnow.h5',  'snowforcing/Barrow-20yrs-maxsnow-30days.h5',
                  'snowforcing/Barrow-20yrs-maxsnow-60days.h5', 'snowforcing/Barrow-20yrs-maxsnow-90days.h5']



infile_forcing = [path + file for file in infile_forcing]


In [3]:
def Read(infile):
    data = dict()
    dat = np.loadtxt(infile,skiprows=6)
    data['snow_depth'] = np.array([t*100 for t in dat[:,1]])
    data['time'] = np.array([(t/86400.) - 264 for t in dat[:,0]])
    loc = np.where(data['snow_depth'][365*5:] > 36.3)[0]
    if len(loc) > 0:
        print (len(loc), 264 + loc[0])
    return data
def Read_forcing(infile,key='precipitation snow [m SWE s^-1]', cumulative=True):
    snow1 = h5py.File(infile,'r')
    data1 = list(snow1[key])[263:523]
    time1 = list(snow1['time [s]'])[263:523]
    
    data1 = np.array([d*100*86400 for d in data1])
    
    #data = dict()
    #data['time'] = np.array([(t/86400.) - 264 for t in time1])
    if cumulative:
        data = np.cumsum(data1)
    else:
        data = data1
    return data

def Read_forcing_temp(infile,key='air temperature [K]'):
    snow1 = h5py.File(infile,'r')
    data1 = list(snow1[key])[263:523]
    data1 = np.array([d*100*86400 for d in data1])
    data = np.cumsum(data1)
    return data

In [4]:
Labels= ['0', '30', '60', '90']
#MaxSnow_day =[489 - 264, 454 -264, 427-264, 398-264] # orig before October 17, 2019
MaxSnow_day =[480 - 264, 454 -264, 427-264, 398-264] # slight shifting in the basecase, max snow depth stay 36 plus/minus 2
print (MaxSnow_day)
#Data_Inset = dict()
DailySnowPrecip = dict()
for i,file in enumerate(infile_forcing):
    key='precipitation snow [m SWE s^-1]'
    DailySnowPrecip[Labels[i]] = Read_forcing(file,key,cumulative=False)
    #Data_Inset[Labels[i]] = data[key]
    
AccuSnowPrecip = dict()
for i,file in enumerate(infile_forcing):
    AccuSnowPrecip[Labels[i]] = Read_forcing(file)

SnowpackDepth = dict()
for i,file in enumerate(infile):
    SnowpackDepth[Labels[i]] = Read(file)

[216, 190, 163, 134]
5 489
50 454
262 427
200 398


In [41]:
MaxSnow_day = []
for key in SnowpackDepth.keys():
    d = SnowpackDepth[key]['snow_depth'][:365]
    #plt.plot(d[0:365])
    for i, x in enumerate(d):
        if x > 36.0:
            print (key, i,x, max(d))
            MaxSnow_day.append(i)
            break
    #print (key, max(d))
    #break

0 214 36.02578461292923 36.29456321990806
30 188 36.06919942077057 36.78360951045303
60 161 36.254172461509086 37.507689630616404
90 131 36.00236137832261 38.72879239261792


In [38]:
import matplotlib.gridspec as gridspec
%matplotlib tk

fig = plt.figure(figsize=(8,6))

#fig,axs = plt.subplots(2,3, figsize=(9,3), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1,1,1]})
fig.subplots_adjust(hspace =.02, wspace=0.02)
gridspec.GridSpec(6,2)
ax1 = plt.subplot2grid((6,2), (0,0), colspan=2, rowspan=3)

plt.tight_layout()
colors = ['k', 'g', 'b', 'r']


key='precipitation snow [m SWE s^-1]'
for i,file in enumerate(infile_forcing[::-1]):
    plt.plot(DailySnowPrecip[Labels[3-i]], color=colors[3-i])
ax1.set_ylim([0,0.8])
ax1.set_xlim([0,253])
ax1.set_xlabel('Time [day]')
ax1.set_ylabel('Snow precipitation [cm]')

#---------------------------------------------------------------------------------------------------------

ax3 = plt.subplot2grid((6,2), (3,0),colspan=1, rowspan=3)
for i,file in enumerate(infile_forcing):
    if i == 0:
        ax3.plot(AccuSnowPrecip[Labels[i]],color=colors[i])#,label=Labels[i])
    else:
        ax3.plot(AccuSnowPrecip[Labels[i]],color=colors[i])#,label=r'S$_{%s}$'%Labels[i])
ax3.set_ylim([0,25])
ax3.set_xlim([0,253])
ax3.set_xlabel('Time [day]')
ax3.set_ylabel('Cumulative snow precipitation [cm]')

ax2 = plt.subplot2grid((6,2), (3,1),colspan=1, rowspan=3)
for i,file in enumerate(infile):
    if i == -1:
        ax2.plot(SnowpackDepth[Labels[i]]['time'], SnowpackDepth[Labels[i]]['snow_depth'], color=colors[i],label=Labels[i])
    else:
        ax2.plot(SnowpackDepth[Labels[i]]['time'], SnowpackDepth[Labels[i]]['snow_depth'], color=colors[i],label=r'S$_{%s}$'%Labels[i])
    ax2.axvline(x=MaxSnow_day[i],ymin=0, ymax=0.73,color=colors[i],linestyle='--')
    print (MaxSnow_day[i])
ax2.set_ylim([0,50])
ax2.set_xlim([0,253])
ax2.set_xlabel('Time [day]')
ax2.set_ylabel('Snowpack depth [cm]')
#ax2.fill_between([0,3650],36.3,38.3,alpha=0.6,color='c') #orig
ax2.fill_between([0,3650],35,39,alpha=0.6,color='gray')
ax2.fill_between([0,3650],37.0001,37.0002,alpha=0.6,color='k',linestyle='--')


ax2.legend(loc='lower right', fontsize=10, ncol=4, bbox_to_anchor=(-0.15, -.45, 1.1, 1))
plt.tight_layout()
plt.savefig(path+'/snowpack_forcing200-xx.pdf', bbox_inches='tight', dpi=100)
print (path)

plt.show()



214
188
161
131
/Users/ajc/Core/Projects/ATS-Data/OR-CONDO/simulations/hillslope/2019/MaxSnow/


In [ ]:
import matplotlib.gridspec as gridspec
%matplotlib tk

fig = plt.figure(1)
#fig,axs = plt.subplots(2,3, figsize=(9,3), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1,1,1]})
#fig.subplots_adjust(hspace =.02, wspace=1.02)
gridspec.GridSpec(4,2)
ax1 = plt.subplot2grid((4,2), (0,0), colspan=2, rowspan=2)

plt.tight_layout()
colors = ['k', 'g', 'b', 'r']



for i,file in enumerate(infile_forcing[::-1]):
    key='precipitation snow [m SWE s^-1]'
    data = Read_forcing(file,key,cumulative=False)
    plt.plot(data[key][::1], color=colors[3-i])
    Data_Inset[Labels[3-i]] = data[key]
ax1.set_ylim([0,0.8])
ax1.set_xlim([0,253])
ax1.set_xlabel('Time [day]')
ax1.set_ylabel('Snow precipitation [mm]')

#---------------------------------------------------------------------------------------------------------

ax3 = plt.subplot2grid((4,2), (2,0),colspan=1, rowspan=2)
for i,file in enumerate(infile_forcing):
    data = Read_forcing(file)
    key='precipitation snow [m SWE s^-1]'
    if i == 0:
        ax3.plot(data[key],color=colors[i])#,label=Labels[i])
    else:
        ax3.plot(data[key],color=colors[i])#,label=r'S$_{%s}$'%Labels[i])
ax3.set_ylim([0,25])
ax3.set_xlim([0,253])
ax3.set_xlabel('Time [day]')
ax3.set_ylabel('Cumulative snow precipitation [cm]')

ax2 = plt.subplot2grid((4,2), (2,1),colspan=1, rowspan=2)
for i,file in enumerate(infile):
    data = Read(file)
    if i == 0:
        ax2.plot(data['time'], data['snow_depth'], color=colors[i],label=Labels[i])
    else:
        ax2.plot(data['time'], data['snow_depth'], color=colors[i],label=r'S$_{%s}$'%Labels[i])
    ax2.axvline(x=MaxSnow_day[i],ymin=0, ymax=0.73,color=colors[i],linestyle='--')
    
ax2.set_ylim([0,50])
ax2.set_xlim([0,253])
ax2.set_xlabel('Time [day]')
ax2.set_ylabel('Snowpack depth [cm]')
ax2.fill_between([0,3650],36.3,38.3,alpha=0.6,color='c')

ax2.legend(loc='lower right', fontsize=11, ncol=4, bbox_to_anchor=(0.1,-0.05, 1.1, 1.1))

#plt.savefig(path+'/snowpack_forcing.pdf', bbox_inches='tight', dpi=100)
fig.tight_layout()
fig.set_size_inches(w=11,h=7)
plt.show()


In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
%matplotlib tk

fig,axs = plt.subplots(2,3, figsize=(9,3), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1,1,1]})
fig.subplots_adjust(hspace =.02, wspace=1.02)
plt.tight_layout()
colors = ['k', 'g', 'b', 'r']
Labels= ['Basecase', '30', '60', '90']
MaxSnow_day =[489 - 264, 454 -264, 427-264, 398-264]
print (MaxSnow_day)
Data_Inset = dict()

ax1 = plt.subplot('131')
for i,file in enumerate(infile_forcing[::-1]):
    key='precipitation snow [m SWE s^-1]'
    data = Read_forcing(file,key,cumulative=False)
    ax1.plot(data[key][::1], color=colors[3-i])
    Data_Inset[Labels[3-i]] = data[key]
ax1.set_ylim([0,0.6])
ax1.set_xlim([0,253])
ax1.set_xlabel('Time [day]')
ax1.set_ylabel('Snowpack depth [cm]')

#---------------------------------------------------------------------------------------------------------
#ax1.fill_between([0,3650],36.3,38.3,alpha=0.6,color='c')

"""
ax2 = plt.subplot('133')
for i,file in enumerate(infile):
    data = Read(file)
    ax2.plot(data['time'], data['snow_depth'], color=colors[i])
    plt.axvline(x=MaxSnow_day[i],ymin=0, ymax=0.73,color=colors[i],linestyle='--')
    
ax2.set_ylim([0,50])
ax2.set_xlim([0,253])
ax2.set_xlabel('Time [day]')
ax2.set_ylabel('Snowpack depth [cm]')
ax2.fill_between([0,3650],36.3,38.3,alpha=0.6,color='c')



ax3 = plt.subplot('132')
for i,file in enumerate(infile_forcing):
    data = Read_forcing(file)
    key='precipitation snow [m SWE s^-1]'
    if i == 0:
        ax3.plot(data[key],color=colors[i],label=Labels[i])
    else:
        ax3.plot(data[key],color=colors[i],label=r'S$_{%s}$'%Labels[i])
ax3.set_ylim([0,25])
ax3.set_xlim([0,253])
ax3.set_xlabel('Time [day]')
ax3.set_ylabel('Cumulative snow precipitation [cm]')

#ax3.legend(loc='upper right', fontsize=11, ncol=4, bbox_to_anchor=(0.9,-0.25, .1, .1))
"""
#plt.savefig(path+'/snowpack_forcing.pdf', bbox_inches='tight', dpi=100)
plt.show()


In [ ]:
ax1 = plt.subplot('111')
for i,file in enumerate(infile_forcing[::-1]):
    key='precipitation snow [m SWE s^-1]'
    
    ax1.plot(Data_Inset[Labels[3-i]], color=colors[3-i])
    #Data_Inset[Labels[3-i]] = data[key]
ax1.set_ylim([0,.8])
ax1.set_xlim([0,253])
ax1.set_xlabel('Time [day]')
ax1.set_ylabel('Snowpack depth [cm]')

axins1 = ax1.inset_axes([0.1, 0.65, 0.3, 0.3])
#axins1 = zoomed_inset_axes(ax1, 1., loc='center',bbox_to_anchor=(150, 250, 10, 10))

axins1.plot(Data_Inset['Basecase'],'r')
axins1.plot(Data_Inset['30'],'k')
axins1.plot(Data_Inset['60'],'c')
axins1.plot(Data_Inset['90'],'b')
axins1.set_xlim(53, 58)
#axins1.set_ylim(0.01, 0.1)

mark_inset(ax1, axins1, loc1=3, loc2=2, fc="none", ec="0.5")
#ax1.indicate_inset_zoom(axins1)
#plt.yticks(visible=False)
#plt.xticks(visible=False)

In [ ]:
M_ = 0.0180153
den = 1000.
#self.Data_ = self.Data_ * self.M_ / (self.liq_den ) * 1.e+6
def ReadHydrograph(file):
    if 'flux' not in file:
        print ('Not hydrograph file: ', file)
        return
    data = dict()
    dat = np.loadtxt(file,skiprows=6)
    #print (len(dat[:,1]))
    dat1 = np.array([t*M_/den * 1.e+6 for t in dat[:,1]])
    # old data['discharge'] = np.reshape(dat1[1:],(-1,4)).mean(axis=1)
    print (len(dat1), len(dat1)%4)
    r=len(dat1)%4
    data['discharge'] = np.reshape(dat1[:-r],(-1,4)).mean(axis=1)
    data['time'] = np.array([(t/86400.)  for t in dat[:,0]])
    
    return data

In [ ]:
DataSim = dict()
Sim = ['B', 'S30', 'S60', 'S90']
for i, file in enumerate(infile):
    #data = rd.ReadSingleFile(file)
    data = ReadHydrograph(file)
    
    DataSim[Sim[i]+'_time'] = data['time']
    DataSim[Sim[i]+'_discharge'] = data['discharge']
    #print (file,len(data['discharge']))
print (list(DataSim))

In [ ]:
%matplotlib tk
fig,axs = plt.subplots(1,1, figsize=(5,3), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1]})
fig.subplots_adjust(hspace =.02, wspace=1.02)
plt.tight_layout()

end = 7300-264
l1,l2 = -365, -1
B = np.cumsum(DataSim['B_discharge'][l1:l2])
S30 = np.cumsum(DataSim['S30_discharge'][l1:l2])
S60 = np.cumsum(DataSim['S60_discharge'][l1:l2])
S90 = np.cumsum(DataSim['S90_discharge'][l1:l2])

ax1 = plt.subplot('111')

plt.plot(B,'r',label=r'S$_{0}$')#'Basecase')
plt.plot(S30,'k',label=r'S$_{30}$')
plt.plot(S60,'c',label=r'S$_{60}$')
plt.plot(S90,'b',label=r'S$_{90}$')
ax1.set_ylabel('Cumulative discharge [cm]')
ax1.set_xlabel('Time [day]')
ax1.set_xlim([140,365])
ax1.legend(loc='lower right', fontsize=9,labelspacing=0.1,bbox_to_anchor=(0.9, -0.01, .1, .1))
# inset axes....
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

axins = zoomed_inset_axes(ax1, 20., loc='upper center',bbox_to_anchor=(130, 90, 10., 10.))
axins.plot(B,'r')
axins.plot(S30,'k')
axins.plot(S60,'c')
axins.plot(S90,'b')
axins.set_xlim(157, 160)
axins.set_ylim(0, 2)
mark_inset(ax1, axins, loc1=2, loc2=4, fc="none", ec="0.5")

axins1 = zoomed_inset_axes(ax1, 6., loc='center',bbox_to_anchor=(250, 100, 60., 60.))
axins1.plot(B,'r')
axins1.plot(S30,'k')
axins1.plot(S60,'c')
axins1.plot(S90,'b')
axins1.set_xlim(354, 364)
axins1.set_ylim(134, 144)

mark_inset(ax1, axins1, loc1=2, loc2=4, fc="none", ec="0.5")
#ax1.set_ylim([120,145])

#ax1.set_xticks(np.linspace(150,365,6))
print (path)
plt.savefig(path+'/discharge_cumm-x.pdf', bbox_inches='tight', dpi=100)
plt.show()

In [ ]:
%matplotlib tk
def Plot_Forcing(infile_loc):
    fig,axs = plt.subplots(1,1, figsize=(4,4), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1]})
    fig.subplots_adjust(hspace =.02, wspace=0.02)
    plt.tight_layout()
    colors = ['k', 'g', 'b', 'r','c','y']
    Labels= ['Basecase', '30', '60', '90']
    MaxSnow_day =[489 - 264, 454 -264, 427-264, 398-264]
    
    ax1 = plt.subplot('111')
    infile_forcing1 = [infile_loc[0],'/Users/ajc/Core/SimDataInputs/forcing_data/Barrow/Barrow-Validation/Barrow-2010_2016-updated-30pc.h5']
    
    data1 = h5py.File(infile_forcing1[1],'r')
    data2 = h5py.File(infile_forcing1[0],'r')
    #data1 = np.array([d-273.15 for ])
    L1= int(len(data1['air temperature [K]'])%365.)
    data_anna = dict()
    data_anna['air temperature [K]'] = data1['air temperature [K]'][:-L1] - 273.15
    
    data_anna['T_air'] = np.reshape(data_anna['air temperature [K]'],(-1,365))

    Years = np.linspace(2010,2015,6)
    
    ax1.plot(data_anna['T_air'][1],color='g',label='2011')
    ax1.plot(data_anna['T_air'][3],color='k',label='2013')
    ax1.plot(data_anna['T_air'][5],color='b',label='2015')
    ax1.plot(data2['air temperature [K]'][-365:]-273.15,color=colors[3],label='Six-year average')
    ax1.axhline(0,linestyle='--',color='gray')
    ax1.set_xlim([0,365])
    ax1.set_ylim([-40,15])
    ax1.set_xlabel('Time [day]')
    ax1.set_ylabel('Air temperature [\u00B0C]')
    
    #ax2 = plt.subplot('122')
    #ax2.legend(loc='upper right', fontsize=11, ncol=4, bbox_to_anchor=(0.9,-0.25, .1, .1))
    ax1.legend(loc='lower center',fontsize=8)
    
    #plt.savefig(path+'/Tair_forcing.pdf', bbox_inches='tight', dpi=100)
    plt.show()

    for d in data_anna['T_air']:
        print ('Avg. T: ', len(d), round(np.mean(d),1))
        
    d1 = data1['precipitation rain [m s^-1]'][:-L1] + data1['precipitation snow [m SWE s^-1]'][:-L1]
    data_anna['precip'] = [x*1000*86400*365. for x in d1]
    data_anna['precip_annual'] = np.reshape(data_anna['precip'],(-1,365))
    for d in data_anna['precip_annual']:
        print ('Avg. Precip: ', len(d), round(np.mean(d),1))

In [ ]:
Plot_Forcing(infile_forcing)